In [768]:
import pandas as pd

In [769]:
df_na = pd.read_csv('final-complete-data-set.csv') # this is the total dataset

/var/folders/3w/9sxng9vs07x5y5pwg23csg4r0000gn/T/ipykernel_35103/786937212.py:1: DtypeWarning: Columns (14,19,26,27,52,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df_na = pd.read_csv('final-complete-data-set.csv')


In [770]:
df_a = pd.read_csv('final-anormal-data-set.csv') # problematic records
df_n = pd.read_csv('final-normal-data-set.csv')  # non-problematic records
df_a['flag'] = 1 # 1 for problem
df_n['flag'] = 0 # 0 for no problem
df = pd.concat([df_a, df_n], axis = 0) # adding the records

/var/folders/3w/9sxng9vs07x5y5pwg23csg4r0000gn/T/ipykernel_35103/2195398428.py:1: DtypeWarning: Columns (14,19,26,27,52,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df_a = pd.read_csv('final-anormal-data-set.csv')


In [771]:
print(df.shape)
print(df.shape)

(95312, 83)
(95312, 83)


In [772]:
list15 = []
for var in df.columns:
    if df[var].isnull().sum()>df.shape[0]*15/100:
        list15.append(var)

In [773]:
list15 # all columns have less than 15% missing values

[]

In [774]:
list0 = []
for var in df.columns:
    if (df[var].isnull().sum()>0):
        list0.append(var)

In [775]:
len(list0) # only 26 columns have some missing value

26

In [776]:
for var in list0:
    df = df[~df[var].isnull()] # dropping these rows

In [777]:
list0 = []
for var in df.columns:
    if (df[var].isnull().sum()>0):
        list0.append(var)
list0 

# there are no more null values now

[]

In [778]:
def vardefiner(df):
    lst = df.columns
    l = []
    l_i = []
    l_f = []
    l_c = []
    l_b = []
    for var in lst:
        if df[var].dtype=='O':
            l_b.append(var)
        elif (len(df[var].unique())<=2)&(df[var].dtype=='O'):
            l_c.append(var)
        elif df[var].dtype=='float':
            l_f.append(var)
        else:
            l_i.append(var)
        
    l = [l_b,l_c,l_f,l_i]
    return(l)


varlist = vardefiner(df) # a function to classify the variables
binary_vars = varlist[0]
cat_vars = varlist[1]
float_vars = varlist[2] # useful for inspecting variables
int_vars = varlist[3]


numlist = int_vars + float_vars
stringlist = binary_vars + cat_vars

In [779]:
for var in numlist: # these columns don't have any information
    if df[var].sum()==0:
        df.drop(var, axis = 1, inplace = True)



for var in stringlist: # dropping variables with only one level
    if len(df[var].unique()) == 1:
        df.drop(var, axis = 1, inplace = True)



In [780]:
varlist = vardefiner(df) # a function to classify the variables
binary_vars = varlist[0]
cat_vars = varlist[1]
float_vars = varlist[2]
int_vars = varlist[3]
numlist = int_vars + float_vars
stringlist = binary_vars + cat_vars

In [781]:
for var in stringlist:
    if len(df[var].unique()) == 1: # these are of practically no use
        df.drop(var, axis = 1, inplace = True)

In [782]:
for var in stringlist:
    print(var, df[var].unique(), df[var].dtypes) 
    
    # there are some numerics in cat_vars

diskio_sda1_time_since_update ['1' '1.0228629112243652' '1.0591349601745603' ... 1.101564884185791
 1.1031060218811035 1.0847859382629397] object
diskio_sda_time_since_update ['1' '1.0228629112243652' '1.0591349601745603' ... 1.101564884185791
 1.1031060218811035 1.0847859382629397] object
fs_/_device_name ['/dev/mapper/centos-root' '/dev/sda1'] object
fs_/_percent ['95.7' '95.8' '95.9' '96' '96.1' '96.7' '96.6' '96.8' '96.9' '95.1'
 '43.4' '43.5' 43.5 34.5 34.6 47.0 47.1 47.2 30.6 30.7 30.8 30.9 23.6 23.7
 23.8 25.5 25.6 25.7] object
fs_/_size ['4708106240' 4708106240 10725883904] object
network_lo_cx ['200' '0' '288' 0 656 288.0] object
network_lo_tx ['100' '0' '144' 0 328 144.0] object
system_hostname ['localhost.localdomain' 'vm1-graph-analytics' 'vm2-node' 'vm3-tomcat'
 'vm4-web-server'] object
system_os_version ['3.10.0-1062.12.1.el7.x86_64' '3.10.0-1062.el7.x86_64'] object
timestamp ['2020-03-20 09:16:03' '2020-03-20 09:16:05' '2020-03-20 09:16:06' ...
 '2020-01-29 22:52:13' '20

In [783]:
stringlist.remove('system_os_version')
stringlist.remove('fs_/_device_name')
stringlist.remove('system_hostname')
stringlist.remove('timestamp') # removing the original cat vars

In [784]:
for var in stringlist:
    df[var] = df[var].astype(float) # converting the remaining variables

In [785]:
varlist = vardefiner(df) # a function to classify the variables
binary_vars = varlist[0]
cat_vars = varlist[1]
float_vars = varlist[2]
int_vars = varlist[3]
numlist = int_vars + float_vars
stringlist = binary_vars + cat_vars

In [786]:
numlist.remove('flag')
cpu = []
diskio = []
mem = []
network = []
load = []
fs = []
processcount = []
for var in numlist:
    if (var[:3] == 'cpu')or(var[3:6]=='cpu'):
        cpu.append(var)
    elif var[:3] == 'mem':
        mem.append(var)
    elif var[:6] == 'diskio':
        diskio.append(var)
    elif var[:7] == 'network':
        network.append(var)
    elif var[:4] == 'load':
        load.append(var)
    elif var[:3] == 'pro':
        processcount.append(var)
    else:
        fs.append(var)  # dividing the numeric types baased on their types

In [787]:
for var in stringlist:
    print(var, df[var].unique(), ) # remaining object types

fs_/_device_name ['/dev/mapper/centos-root' '/dev/sda1']
system_hostname ['localhost.localdomain' 'vm1-graph-analytics' 'vm2-node' 'vm3-tomcat'
 'vm4-web-server']
system_os_version ['3.10.0-1062.12.1.el7.x86_64' '3.10.0-1062.el7.x86_64']
timestamp ['2020-03-20 09:16:03' '2020-03-20 09:16:05' '2020-03-20 09:16:06' ...
 '2020-01-29 22:52:13' '2020-01-29 22:52:14' '2020-01-29 22:52:15']


In [788]:
from datetime import datetime

#we will see how certain variables act over the timeline

df['datetime'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df.shape

(95310, 61)

In [789]:
df.drop('timestamp', axis = 1, inplace = True)

In [790]:
df['datetime']

0       2020-03-20 09:16:03
1       2020-03-20 09:16:05
2       2020-03-20 09:16:06
3       2020-03-20 09:16:07
4       2020-03-20 09:16:08
                ...        
80846   2020-01-29 22:52:11
80847   2020-01-29 22:52:12
80848   2020-01-29 22:52:13
80849   2020-01-29 22:52:14
80850   2020-01-29 22:52:15
Name: datetime, Length: 95310, dtype: datetime64[ns]

In [791]:
binary_vars.remove('timestamp')
stringlist.remove('timestamp')

## Inspection

### Load Variables

In [792]:
df[load].describe()

,load_cpucore,load_min1,load_min15,load_min5
count,95310.000000,95310.000000,95310.000000,95310.000000
mean,1.045210,1.439928,1.348724,1.416121
std,0.207766,1.466245,1.374236,1.432308
min,1.000000,0.000000,0.050000,0.020000
25%,1.000000,0.180000,0.210000,0.190000
50%,1.000000,0.520000,0.270000,0.340000
75%,1.000000,3.010000,3.100000,3.120000
max,2.000000,5.550000,4.200000,4.510000


In [793]:

df_lu = df[df['load_cpucore']>1]

df[df['load_cpucore']>1].load_min1.describe()

count    4309.000000
mean        3.680733
std         0.412826
min         0.960000
25%         3.600000
50%         3.730000
75%         3.860000
max         4.650000
Name: load_min1, dtype: float64

In [794]:
df_lu[load].describe()

,load_cpucore,load_min1,load_min15,load_min5
count,4309.0,4309.000000,4309.000000,4309.000000
mean,2.0,3.680733,3.135024,3.545523
std,0.0,0.412826,0.899465,0.621365
min,2.0,0.960000,0.180000,0.420000
25%,2.0,3.600000,2.890000,3.670000
50%,2.0,3.730000,3.600000,3.740000
75%,2.0,3.860000,3.700000,3.790000
max,2.0,4.650000,3.800000,4.000000


In [795]:
df_ld = df[df['load_cpucore']<=1]

In [796]:
df_ld[load].describe()

,load_cpucore,load_min1,load_min15,load_min5
count,91001.0,91001.000000,91001.000000,91001.000000
mean,1.0,1.333823,1.264140,1.315291
std,0.0,1.412298,1.334691,1.380397
min,1.0,0.000000,0.050000,0.020000
25%,1.0,0.170000,0.200000,0.190000
50%,1.0,0.410000,0.250000,0.300000
75%,1.0,2.740000,2.550000,2.960000
max,1.0,5.550000,4.200000,4.510000


### CPU feature Variables

In [797]:
df[cpu].describe()

,cpu_idle,cpu_iowait,cpu_nice,cpu_softirq,cpu_system,cpu_total,cpu_user,percpu_0_idle,percpu_0_iowait,percpu_0_nice,percpu_0_softirq,percpu_0_system,percpu_0_total,percpu_0_user
count,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000,95310.000000
mean,50.092828,0.003336,0.004094,0.068612,3.348626,49.934123,39.904601,50.047247,0.003525,0.004084,0.054764,3.317834,49.952753,39.124444
std,45.256183,0.057063,0.466298,0.271454,1.521660,45.247559,41.977910,45.259543,0.068033,0.466377,0.234905,1.553371,45.259543,41.011366
min,0.000000,0.000000,0.000000,0.000000,0.000000,3.200000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.300000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,2.500000,8.300000,5.500000,0.000000,0.000000,0.000000,0.000000,2.500000,8.300000,5.500000
50%,88.900000,0.000000,0.000000,0.000000,2.800000,11.100000,7.300000,88.900000,0.000000,0.000000,0.000000,2.800000,11.100000,7.300000
75%,91.700000,0.000000,0.000000,0.000000,3.700000,100.000000,95.200000,91.700000,0.000000,0.000000,0.000000,3.700000,100.000000,94.500000
max,94.300000,3.600000,75.500000,3.000000,48.000000,100.000000,99.200000,96.700000,10.000000,74.500000,3.000000,26.800000,100.000000,100.000000


In [798]:
df[df['cpu_user']>=95].shape

(25439, 60)

In [799]:
df[df['cpu_iowait']>0].shape

(344, 60)

In [800]:
df[df['cpu_nice']>0].shape

(59, 60)

In [801]:
df[df['cpu_softirq']>0].shape

(6310, 60)

#### The CPU variables are devided into two matrices, cpu and percpu
- Each of them have these variables
  - idle 
  - iowait* 
  - nice*
  - softirq*
  - system
  - total
  - user
- In these the starred variables are irksome, because
  - few of the rows display suspicious behaviour, meaning they are not statistically regular with the remaining data
- We will flag the records that show these divergent tendencies

In [802]:
cpuflags = ['cpu_iowait','cpu_nice','cpu_softirq','percpu_0_iowait','percpu_0_nice','percpu_0_softirq']

In [803]:
def flaglist(x):
    if x>0:
        return 1
    else:
        return 0

In [804]:
for var in cpuflags:
    df[var+'_flag'] = df[var].apply(flaglist)

### DISKio variables

In [805]:
df[diskio].describe()# we see the same tendencies here aswell

,diskio_sda1_write_bytes,diskio_sda_write_bytes,diskio_sda1_read_bytes,diskio_sda1_time_since_update,diskio_sda_read_bytes,diskio_sda_time_since_update
count,9.531000e+04,9.531000e+04,95310.000000,95310.000000,9.531000e+04,95310.000000
mean,1.925485e+04,2.341422e+04,6.403357,1.162641,2.191023e+03,1.162641
std,1.212751e+05,1.887162e+05,1481.391724,0.121199,1.378945e+05,0.121199
min,0.000000e+00,0.000000e+00,0.000000,0.652700,0.000000e+00,0.652700
25%,0.000000e+00,0.000000e+00,0.000000,1.087129,0.000000e+00,1.087129
50%,0.000000e+00,0.000000e+00,0.000000,1.093017,0.000000e+00,1.093017
75%,0.000000e+00,0.000000e+00,0.000000,1.209975,0.000000e+00,1.209975
max,3.051520e+06,3.473408e+07,454656.000000,2.127072,1.922662e+07,2.127072


In [806]:
df[df['diskio_sda1_write_bytes']>7*(10**4)].shape

(3984, 66)

#### Problem Variables
- diskio_sda1_write_bytes
- diskio_sda1_write_bytes
- diskio_sda1_read_bytes
- diskio_sda_read_bytes

In [807]:
ioflags = ['diskio_sda1_write_bytes','diskio_sda1_write_bytes']
ioflags1 = ['diskio_sda1_read_bytes','diskio_sda_read_bytes']

In [808]:
dfx = df[df['diskio_sda1_write_bytes']>0]

In [809]:
dfx[diskio].describe()

,diskio_sda1_write_bytes,diskio_sda_write_bytes,diskio_sda1_read_bytes,diskio_sda1_time_since_update,diskio_sda_read_bytes,diskio_sda_time_since_update
count,1.631100e+04,1.631100e+04,16311.000000,16311.000000,16311.000000,16311.000000
mean,1.125118e+05,1.125118e+05,9.291398,1.201828,9.291398,1.201828
std,2.746860e+05,2.746860e+05,386.092388,0.142082,386.092388,0.142082
min,4.096000e+03,4.096000e+03,0.000000,1.074106,0.000000,1.074106
25%,1.228800e+04,1.228800e+04,0.000000,1.087478,0.000000,1.087478
50%,2.457600e+04,2.457600e+04,0.000000,1.096492,0.000000,1.096492
75%,6.553600e+04,6.553600e+04,0.000000,1.356405,0.000000,1.356405
max,3.051520e+06,3.051520e+06,16384.000000,1.679426,16384.000000,1.679426


In [810]:
df[df['diskio_sda1_read_bytes']>0].shape

(12, 66)

In [811]:
for var in ioflags:
    df[var+'_flag'] = df[var].apply(flaglist)

In [812]:
def flaglist1(x):
    if x>7*(10**4):
        return 1
    else:
        return 0

In [813]:
for var in ioflags1:
    df[var+'_flag'] = df[var].apply(flaglist1)

### Memory Variables

In [814]:
df[mem].describe()

,mem_active,mem_available,mem_cached,mem_free,mem_inactive,mem_used,memswap_free,memswap_sin,memswap_sout,memswap_total,memswap_used,mem_buffers,mem_percent,mem_shared,mem_total,memswap_percent
count,9.531000e+04,9.531000e+04,9.531000e+04,9.531000e+04,9.531000e+04,9.531000e+04,9.531000e+04,95310.000000,9.531000e+04,9.531000e+04,9.531000e+04,9.531000e+04,95310.000000,9.531000e+04,9.531000e+04,95310.000000
mean,7.304257e+08,2.643991e+09,1.126026e+09,2.643991e+09,4.683325e+08,1.014084e+09,9.789841e+07,1407.621278,8.791848e+04,9.799585e+07,9.744733e+04,1.515775e+06,28.036302,8.841971e+06,3.658076e+09,0.013342
std,4.362575e+08,8.222815e+08,5.762290e+08,8.222815e+08,2.604473e+08,8.028522e+08,2.314921e+08,10478.797410,7.899621e+05,2.317212e+08,8.038916e+05,9.426470e+05,20.663529,3.821109e+05,4.773769e+08,0.120408
min,1.990943e+08,3.805512e+08,3.616645e+08,3.805512e+08,1.708851e+08,4.581171e+08,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,11.500000,6.938624e+06,1.039180e+09,0.000000
25%,5.265009e+08,2.584774e+09,7.167437e+08,2.584774e+09,3.329393e+08,6.605169e+08,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.679360e+05,17.800000,8.863744e+06,3.709407e+09,0.000000
50%,5.646090e+08,3.031523e+09,8.777667e+08,3.031523e+09,3.329556e+08,6.775767e+08,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.117632e+06,18.300000,8.896512e+06,3.709415e+09,0.000000
75%,6.282230e+08,3.048616e+09,1.221140e+09,3.048616e+09,5.456445e+08,8.820214e+08,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.117632e+06,30.300000,8.962048e+06,3.709415e+09,0.000000
max,2.037846e+09,3.515314e+09,2.226229e+09,3.515314e+09,9.435546e+08,3.593052e+09,6.459187e+08,98304.000000,7.217152e+06,6.459187e+08,7.340032e+06,2.158592e+06,90.400000,9.072640e+06,3.973603e+09,1.100000


In [815]:
df[df['memswap_percent']>0].shape

(1156, 69)

In [816]:
df[df['memswap_total']>0]

,cpu_idle,cpu_iowait,cpu_nice,cpu_softirq,cpu_system,cpu_total,cpu_user,diskio_sda1_read_bytes,diskio_sda1_time_since_update,diskio_sda1_write_bytes,...,datetime,cpu_iowait_flag,cpu_nice_flag,cpu_softirq_flag,percpu_0_iowait_flag,percpu_0_nice_flag,percpu_0_softirq_flag,diskio_sda1_write_bytes_flag,diskio_sda1_read_bytes_flag,diskio_sda_read_bytes_flag
0,0.0,0.0,0.0,0.0,2.0,100.0,6.0,0.0,1.000000,0,...,2020-03-20 09:16:03,0,0,0,0,0,1,0,0,0
1,0.0,0.0,0.0,0.0,2.0,100.0,39.0,0.0,1.022863,0,...,2020-03-20 09:16:05,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,2.0,100.0,42.0,0.0,1.059135,0,...,2020-03-20 09:16:06,0,0,0,0,0,0,0,0,1
3,0.0,0.0,0.0,0.0,2.0,100.0,40.0,0.0,0.992989,0,...,2020-03-20 09:16:07,0,0,0,0,0,0,0,0,1
4,0.0,0.0,0.0,0.0,1.0,100.0,37.0,0.0,1.007005,0,...,2020-03-20 09:16:08,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14456,0.0,0.0,1.0,0.0,1.0,100.0,32.0,0.0,1.272906,0,...,2020-01-28 15:31:27,0,1,0,0,1,0,0,0,0
14457,0.0,0.0,0.0,0.0,2.0,100.0,36.0,0.0,1.302468,0,...,2020-01-28 15:31:28,0,0,0,0,0,0,0,0,0
14458,0.0,0.0,0.0,0.0,2.0,100.0,39.0,0.0,1.267522,0,...,2020-01-28 15:31:29,0,0,0,0,0,0,0,0,0
14459,0.0,0.0,0.0,0.0,1.0,100.0,34.0,0.0,1.276569,0,...,2020-01-28 15:31:30,0,0,0,0,0,0,0,0,0


In [817]:
memoryflags = ['memswap_free', 'memswap_sin', 'memswap_sout', 'memswap_total', 'memswap_used', 'memswap_percent']

#### Problem Variables
- memswap_free
- memswap_sin
- memswap_sout
- memswap_total
- memswap_used
- memswap_percent

In [818]:
for var in memoryflags:
    df[var+'_flag'] = df[var].apply(flaglist)

### Network Variables

In [819]:
df[network].describe()

,network_lo_cumulative_cx,network_lo_cumulative_rx,network_lo_cumulative_tx,network_lo_cx,network_lo_rx,network_lo_time_since_update,network_lo_tx
count,95310.000000,95310.000000,95310.000000,95310.00000,95310.000000,95310.000000,95310.000000
mean,31780.512055,15890.256028,15890.256028,2.28039,1.140195,1.162641,1.140195
std,42494.928252,21247.464126,21247.464126,25.56800,12.784000,0.121031,12.784000
min,400.000000,200.000000,200.000000,0.00000,0.000000,0.695460,0.000000
25%,12952.000000,6476.000000,6476.000000,0.00000,0.000000,1.087103,0.000000
50%,18464.000000,9232.000000,9232.000000,0.00000,0.000000,1.092982,0.000000
75%,19940.000000,9970.000000,9970.000000,0.00000,0.000000,1.209952,0.000000
max,194680.000000,97340.000000,97340.000000,656.00000,328.000000,2.114878,328.000000


In [820]:
df[df['network_lo_cumulative_cx']>100000].shape

(9525, 75)

In [821]:
df[df['network_lo_cumulative_rx']>70000].shape

(5457, 75)

In [822]:
df[df['network_lo_tx']>0].shape

(754, 75)

In [823]:
def flaglist2(x):
    if x>10**5:
        return 1
    else:
        return 0

In [824]:
networkflag1 = ['network_lo_cumulative_cx']
networkflag2 = ['network_lo_cumulative_rx', 'network_lo_cumulative_tx']
networkflag3 = ['network_lo_cx', 'network_lo_rx', 'network_lo_tx']

In [825]:
for var in network:
    if var in networkflag1:
        df[var+'_flag'] = df[var].apply(flaglist2)
    elif var in networkflag2:
        df[var+'_flag'] = df[var].apply(flaglist1)
    else:
        df[var+'_flag'] = df[var].apply(flaglist)

#### Problem Variables
- network_lo_cumulative_cx
- network_lo_cumulative_rx
- network_lo_cumulative_tx
- network_lo_cx
- network_lo_rx
- network_lo_tx

### FS variables

In [826]:
df[fs].describe() 

,fs_/_free,fs_/_used,fs_/_percent,fs_/_size
count,9.531000e+04,9.531000e+04,95310.000000,9.531000e+04
mean,6.458996e+09,3.353898e+09,37.059822,9.812894e+09
std,2.327729e+09,1.019461e+09,18.979095,2.158858e+09
min,1.453998e+08,1.622942e+09,23.600000,4.708106e+09
25%,5.671690e+09,2.735157e+09,25.600000,1.072588e+10
50%,7.429366e+09,2.755858e+09,30.700000,1.072588e+10
75%,7.983272e+09,4.551557e+09,47.100000,1.072588e+10
max,8.194929e+09,5.067731e+09,96.900000,1.072588e+10


In [827]:
df[df['fs_/_percent']>50].shape

(7182, 82)

In [828]:
df['fs_/_percent_flag'] = df['fs_/_percent'].apply(lambda x: 1 if x>50 else 0)

#### Problem Variable
- fs_/_percent

### Processcount Variables

In [829]:
df[processcount].describe()

,processcount_running,processcount_sleeping,processcount_thread,processcount_total
count,95310.000000,95310.000000,95310.000000,95310.000000
mean,2.063456,99.633218,147.371504,101.695226
std,0.445196,7.598300,17.898651,7.615595
min,1.000000,73.000000,125.000000,1.000000
25%,2.000000,94.000000,138.000000,96.000000
50%,2.000000,98.000000,142.000000,100.000000
75%,2.000000,106.000000,153.000000,109.000000
max,95.000000,122.000000,211.000000,124.000000


In [830]:
df[df['processcount_running']>2].shape

(7463, 83)

In [831]:
df['processcount_running_flag'] = df['processcount_running'].apply(lambda x: 1 if x>2 else 0)

#### Problem Variable
- processcount_running

## Now that we have flagged every statistically unreliable behaviour

In [832]:
s = 'processcount_running_flag'
s[-4:]

'flag'

In [833]:
flagvars = []
for var in df.columns:
    if var[-4:]=='flag':
        flagvars.append(var)

In [834]:
flagvars

['flag',
 'cpu_iowait_flag',
 'cpu_nice_flag',
 'cpu_softirq_flag',
 'percpu_0_iowait_flag',
 'percpu_0_nice_flag',
 'percpu_0_softirq_flag',
 'diskio_sda1_write_bytes_flag',
 'diskio_sda1_read_bytes_flag',
 'diskio_sda_read_bytes_flag',
 'memswap_free_flag',
 'memswap_sin_flag',
 'memswap_sout_flag',
 'memswap_total_flag',
 'memswap_used_flag',
 'memswap_percent_flag',
 'network_lo_cumulative_cx_flag',
 'network_lo_cumulative_rx_flag',
 'network_lo_cumulative_tx_flag',
 'network_lo_cx_flag',
 'network_lo_rx_flag',
 'network_lo_time_since_update_flag',
 'network_lo_tx_flag',
 'fs_/_percent_flag',
 'processcount_running_flag']

In [835]:
for var in flagvars:
    print(var, df[var].value_counts())

flag 0    80850
1    14460
Name: flag, dtype: int64
cpu_iowait_flag 0    94966
1      344
Name: cpu_iowait_flag, dtype: int64
cpu_nice_flag 0    95251
1       59
Name: cpu_nice_flag, dtype: int64
cpu_softirq_flag 0    89000
1     6310
Name: cpu_softirq_flag, dtype: int64
percpu_0_iowait_flag 0    94963
1      347
Name: percpu_0_iowait_flag, dtype: int64
percpu_0_nice_flag 0    95251
1       59
Name: percpu_0_nice_flag, dtype: int64
percpu_0_softirq_flag 0    90190
1     5120
Name: percpu_0_softirq_flag, dtype: int64
diskio_sda1_write_bytes_flag 0    78999
1    16311
Name: diskio_sda1_write_bytes_flag, dtype: int64
diskio_sda1_read_bytes_flag 0    95309
1        1
Name: diskio_sda1_read_bytes_flag, dtype: int64
diskio_sda_read_bytes_flag 0    95180
1      130
Name: diskio_sda_read_bytes_flag, dtype: int64
memswap_free_flag 0    80850
1    14460
Name: memswap_free_flag, dtype: int64
memswap_sin_flag 0    91185
1     4125
Name: memswap_sin_flag, dtype: int64
memswap_sout_flag 0    91185
1

In [836]:
df[flagvars].Totalflags = df[flagvars].sum(axis = 1, numeric_only = True)

/var/folders/3w/9sxng9vs07x5y5pwg23csg4r0000gn/T/ipykernel_35103/4242909027.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df[flagvars].Totalflags = df[flagvars].sum(axis = 1, numeric_only = True)


In [837]:
df['totalflags'] = 0 # let's count the total flags
for var in flagvars:
    df['totalflags'] = df['totalflags'] +df[var]

In [838]:
df['totalflags'].describe()

count    95310.000000
mean         2.289875
std          2.061385
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         13.000000
Name: totalflags, dtype: float64

In [839]:
df[df['totalflags']>4].shape # more number of flag means higher risk of cryptojaacking

(16832, 85)

In [840]:
df['an_flag'] = df['totalflags'].apply(lambda x: 1 if x>4 else 0) # these records are reaally at threat

# according to my analysis

In [845]:
from sklearn.metrics import accuracy_score, classification_report

# let's see how many threats have been correctly classified

In [847]:
print(classification_report(df.flag, df.an_flag)) # wow

# the thieves have been caught

              precision    recall  f1-score   support

           0       1.00      0.97      0.98     80850
           1       0.86      1.00      0.92     14460

    accuracy                           0.97     95310
   macro avg       0.93      0.98      0.95     95310
weighted avg       0.98      0.97      0.97     95310



In [848]:
df.shape

(95310, 86)

In [849]:
df.to_csv('analysed-final_data.csv')